# !IMPORTANTE: 
    debido a que muchos datos no se encuentran en condiciones de ser graficados se contara  con un  segundo archivo EDA_Parte2 donde se realizara tanto el analisis estadistico como el analisis de correlacion bi y multivariado

In [115]:
import numpy as np
import pandas as pd
import seaborn as sns
import ast
from matplotlib import pyplot as plt
import ETL_functions as etf

<hr>

## EDA "steam_games"

<hr>

### cargando datos y primeras observaciones

In [74]:
df_sg = pd.read_json(r"../Datasets/steam_games.json/output_steam_games.json", lines=True)#cargando el archivo

In [75]:
df_sg.shape # dimensiones (filas,columnas) 

(120445, 13)

In [88]:
df_sg.sample(5)#ejecute esta linea aprox(5) veces para tener una vision mas amplia de algunos datos aleatorios del df 
               #notando que hay muchas filas llenas de valores None o NaN
               #a demas hay columnas que contienen listas como valores

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
108902,Big Fish Games,"[Adventure, Casual]",Rite of Passage: The Perfect Show Collector's ...,Rite of Passage: The Perfect Show Collector's ...,http://store.steampowered.com/app/552670/Rite_...,2016-11-11,"[Adventure, Casual, Hidden Object]",http://steamcommunity.com/app/552670/reviews/?...,[Single-player],9.99,0.0,552670.0,Mad Head Games
109805,Good Games,"[Free to Play, Strategy]",Galaxy Reavers: Cruiser DLC,Galaxy Reavers: Cruiser DLC,http://store.steampowered.com/app/505650/Galax...,2016-09-19,"[Free to Play, Strategy]",http://steamcommunity.com/app/505650/reviews/?...,"[Single-player, Downloadable Content, Steam Ac...",Free To Play,0.0,505650.0,Good Games
76926,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
63484,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
107254,Degica,"[Design &amp; Illustration, Web Publishing]",RPG Maker MV - Hiroki Kikuta music pack: The Calm,RPG Maker MV - Hiroki Kikuta music pack: The Calm,http://store.steampowered.com/app/599441/RPG_M...,2017-02-27,"[Design & Illustration, Web Publishing]",http://steamcommunity.com/app/599441/reviews/?...,"[Downloadable Content, Steam Trading Cards, Pa...",14.99,0.0,599441.0,"KADOKAWA,Yoji Ojima,Hiroki Kikuta"


### tipos de datos

In [102]:
df_sg.info()#podemos notar que solo contamos con 2 columnas numericas (id,early_access) siendo inecesario analizar con un describe estas
            #podemos ver ademas que early access es float cuando deberia ser un bool(true o false){aclarado en el diccionario} o int (0 o 1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   publisher     24083 non-null   object 
 1   genres        120445 non-null  object 
 2   app_name      32133 non-null   object 
 3   title         30085 non-null   object 
 4   url           32135 non-null   object 
 5   release_date  30068 non-null   object 
 6   tags          120445 non-null  object 
 7   reviews_url   32133 non-null   object 
 8   specs         120445 non-null  object 
 9   price         30758 non-null   object 
 10  early_access  32135 non-null   float64
 11  id            32133 non-null   float64
 12  developer     28836 non-null   object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


In [110]:
# podemos notar que la columna price no se encuentra en float esto puede deberse a que hay valores no convertibles
# para verificar esto e creado una funcion que me ayudara cuando deba corregir esto en un archivo llamado ETL_functions.py el cual importe como etf
etf.valores_no_numericos(df_sg["price"])

['Free To Play',
 'Free to Play',
 'Free',
 'Free Demo',
 'Play for Free!',
 'Install Now',
 'Play WARMACHINE: Tactics Demo',
 'Free Mod',
 'Install Theme',
 'Third-party',
 'Play Now',
 'Free HITMAN™ Holiday Pack',
 'Play the Demo',
 'Starting at $499.00',
 'Starting at $449.00',
 'Free to Try',
 'Free Movie',
 'Free to Use']

### valores nulos

In [107]:
df_sg.isna().sum() #estos son los campos con valores nulos y su cantidad de nulos

publisher       96362
genres              0
app_name        88312
title           90360
url             88310
release_date    90377
tags                0
reviews_url     88312
specs               0
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

### valores duplicados

In [98]:
#tuve que cambiar el tipo de datos a str por que tenian listas como valor  y no me permitian validar duplicados
df_sg["genres"] = df_sg["genres"].astype(str)
df_sg["tags"] = df_sg["tags"].astype(str)
df_sg["specs"] = df_sg["specs"].astype(str)

In [99]:
df_sg.duplicated().sum()# cantidad de registros duplicados

88309

In [100]:
120445 - 88309 # filas totales - filas duplicadas = aproximacion de (datos a trabajar)
               # tener en cuenta que al borrar duplicados aun asi nos quedara un registro
               # unico de las filas que tenian todos sus valores en null la cual deberemos retirar

32136

<hr>

## EDA "user_reviews"

### cargando datos y primeras observaciones

In [151]:
#carga de el archivo
reviews = []
with open(r"../Datasets/user_reviews.json/australian_user_reviews.json", 'r', encoding='utf-8') as f:
    # loop for insert data into list
    for line in f.readlines():
        reviews.append(ast.literal_eval(line))
df_ur = pd.DataFrame(reviews)

In [156]:
df_ur.sample(5)# repetimos el sample varias veces
                # notamos que la columna reviews contiene listas
                # y que user id contiene tanto numeros como cadenas (cosa que puede que este bien por lo que no vamos a profundizar)

,user_id,user_url,reviews
16131,Jac95501,http://steamcommunity.com/id/Jac95501,"[{'funny': '', 'posted': 'Posted January 9, 20..."
8949,76561198061357639,http://steamcommunity.com/profiles/76561198061...,"[{'funny': '1 person found this review funny',..."
5781,Roanoke,http://steamcommunity.com/id/Roanoke,"[{'funny': '', 'posted': 'Posted October 26, 2..."
24205,76561198090621006,http://steamcommunity.com/profiles/76561198090...,"[{'funny': '', 'posted': 'Posted April 29.', '..."
21793,76561198067371563,http://steamcommunity.com/profiles/76561198067...,"[{'funny': '', 'posted': 'Posted December 19, ..."


### tipos de datos
|columna |tipo|
|--------|----|
|user_id | str|
|user_url| str|
|reviews |list|

### valores nulos

In [163]:
df_ur.isna().sum() #estos son los campos con valores nulos y su cantidad de nulos  al parecer no hay valores nulos pero podriaa haber reviews que contengan listas vacias

user_id     0
user_url    0
reviews     0
dtype: int64

### valores duplicados

In [160]:
df_ur["reviews"] = df_ur["reviews"].astype(str) #paso la columna a str para poder contar los registros dupliados

In [161]:
df_ur.duplicated().sum() # hay 313 registros duplicados lo que no nos interesa y los borraremos en el ETL

313

<hr>

## EDA "users_items"

### cargando datos y primeras observaciones

In [165]:
items = []

with open(r'../Datasets/users_items.json/australian_users_items.json', 'r', encoding='utf-8') as f:
    for line in f.readlines():
        items.append(ast.literal_eval(line))

df_ui = pd.DataFrame(items)

In [182]:
df_ui.sample(5)#repetimos el sample varias veces

,user_id,items_count,steam_id,user_url,items
53468,76561198048592484,7,76561198048592484,http://steamcommunity.com/profiles/76561198048...,"[{'item_id': '550', 'item_name': 'Left 4 Dead ..."
36230,za3n,21,76561198070028713,http://steamcommunity.com/id/za3n,"[{'item_id': '220', 'item_name': 'Half-Life 2'..."
70774,TheSheep498,113,76561198075495257,http://steamcommunity.com/id/TheSheep498,"[{'item_id': '105400', 'item_name': 'Fable III..."
84208,76561198099145359,0,76561198099145359,http://steamcommunity.com/profiles/76561198099...,[]
56911,76561198057678203,0,76561198057678203,http://steamcommunity.com/profiles/76561198057...,[]


## tipos de datos

|columna|tipo|
|-------|----|
|user_id|str|
|item_counts|int|
|steam_id|int|
|user_url|str|
|item|list|

In [184]:
df_ui.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88310 entries, 0 to 88309
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      88310 non-null  object
 1   items_count  88310 non-null  int64 
 2   steam_id     88310 non-null  object
 3   user_url     88310 non-null  object
 4   items        88310 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.4+ MB


In [185]:
#steam_id no se encuentra en int pero al parecer no tiene valores no numericos solo se debe a que tiene numeros muy grandes y el tipo de dato de pandas debe ser un dtype que pueda contener  un numero asi de grande 
etf.valores_no_numericos(df_ui["steam_id"])

[]

### valores nulos

In [189]:
df_ui.isna().sum() #estos son los campos con valores nulos y su cantidad de nulos dice no tener valores nulos pero items puede contener listas vacias

user_id        0
items_count    0
steam_id       0
user_url       0
items          0
dtype: int64

### valores duplicados

In [187]:
df_ui["items"] = df_ui["items"].astype(str)#convertirmos a str para poder verificar duplicados

In [188]:
df_ui.duplicated().sum()#tiene 657 que los borrare

657